In [44]:
%reload_ext autoreload
%autoreload 2

from transformers import BertConfig, BertModel
import torch

import sys 
sys.path.append('../')

from train import get_data, get_opt, construct_hyper_param, get_models
from sqlova.utils.utils_wikisql import get_fields, get_g, get_g_wvi_corenlp, get_wemb_bert, \
                                       get_g_wvi_bert_from_g_wvi_corenlp, generate_inputs
import argparse

from tqdm.auto import tqdm

In [28]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [29]:
parser = argparse.ArgumentParser()
args = construct_hyper_param(parser, notebook=True)
args.bS = 8

BERT-type: uncased_L-12_H-768_A-12


In [30]:
path_h = '../data/ko_token_not_h' 
path_wikisql = '../data/ko_token_not_h'  
BERT_PT_PATH = path_wikisql

# Load Data

In [31]:
train_data, train_table, dev_data, dev_table, train_loader, dev_loader = get_data(path_wikisql, args)

# Load Model

In [32]:
path_model_bert = '../model_bert_best.pt'
path_model = '../model_best.pt'

In [33]:
args.bert_name = 'bert-base-multilingual-cased'

In [34]:
# To start from the pre-trained models, un-comment following lines.

model, model_bert, tokenizer, bert_config = get_models(args, BERT_PT_PATH, trained=False,
                                                       path_model_bert=path_model_bert, path_model=path_model,
                                                       device=device)

Batch_size = 8
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001


In [35]:
opt, opt_bert = get_opt(args, model, model_bert, args.fine_tune)

# Train example

In [21]:
t = next(iter(train_loader))

In [36]:
t = next(iter(dev_loader))

In [46]:
for idx, t in enumerate(tqdm(dev_loader)):
    nlu, nlu_t, sql_i, sql_q, sql_t, tb, hs_t, hds = get_fields(t, dev_table, no_hs_t=True, no_sql_t=True)

    l_n = []
    l_hs = []  # The length of columns for each batch

    input_ids = []
    tokens = []
    segment_ids = []
    input_mask = []

    i_nlu = []  # index to retreive the position of contextual vector later.
    i_hds = []

    doc_tokens = []
    nlu_tt = []

    t_to_tt_idx = []
    tt_to_t_idx = []

    for b, nlu_t1 in enumerate(nlu_t):

        hds1 = hds[b]
        l_hs.append(len(hds1))


        # 1. 2nd tokenization using WordPiece
        tt_to_t_idx1 = []  # number indicates where sub-token belongs to in 1st-level-tokens (here, CoreNLP).
        t_to_tt_idx1 = []  # orig_to_tok_idx[i] = start index of i-th-1st-level-token in all_tokens.
        nlu_tt1 = []  # all_doc_tokens[ orig_to_tok_idx[i] ] returns first sub-token segement of i-th-1st-level-token
        for (i, token) in enumerate(nlu_t1):
            t_to_tt_idx1.append(
                len(nlu_tt1))  # all_doc_tokens[ indicate the start position of original 'white-space' tokens.
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tt_to_t_idx1.append(i)
                nlu_tt1.append(sub_token)  # all_doc_tokens are further tokenized using WordPiece tokenizer
        nlu_tt.append(nlu_tt1)
        tt_to_t_idx.append(tt_to_t_idx1)
        t_to_tt_idx.append(t_to_tt_idx1)

        l_n.append(len(nlu_tt1))
        #         hds1_all_tok = tokenize_hds1(tokenizer, hds1)



        # [CLS] nlu [SEP] col1 [SEP] col2 [SEP] ...col-n [SEP]
        # 2. Generate BERT inputs & indices.
        tokens1, segment_ids1, i_nlu1, i_hds1 = generate_inputs(tokenizer, nlu_tt1, hds1)
        input_ids1 = tokenizer.convert_tokens_to_ids(tokens1)

        # Input masks
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask1 = [1] * len(input_ids1)

        # 3. Zero-pad up to the sequence length.
        while len(input_ids1) < args.max_seq_length:
            input_ids1.append(0)
            input_mask1.append(0)
            segment_ids1.append(0)

        assert len(input_ids1) == args.max_seq_length, f'IDX: {idx}'
        assert len(input_mask1) == args.max_seq_length
        assert len(segment_ids1) == args.max_seq_length
    

AssertionError: IDX: 972

In [48]:
for idx, t in enumerate(dev_loader):
    if idx == 972:
        temp = t

In [49]:
temp

[{'table_id': '2-12394513-1',
  'phase': 2,
  'question': 'ketftv.com의 해상도와 kldo-dt2의 콜사인 이름',
  'question_tok': ['ketftv',
   '.',
   'com',
   '의',
   '해상도',
   '와',
   'kldo',
   '-',
   'dt',
   '2',
   '의',
   '콜사',
   '인',
   '이름'],
  'sql': {'sel': 3,
   'conds': [[5, 0, 'ketftv.com'], [1, 0, 'kldo-dt2']],
   'agg': 0},
  'query': {'sel': 3,
   'conds': [[5, 0, 'ketftv.com'], [1, 0, 'kldo-dt2']],
   'agg': 0},
  'wvi_corenlp': [[0, 2], [6, 9]]},
 {'table_id': '2-12394513-1',
  'phase': 2,
  'question': '8126의 접시로 해상도 이름',
  'question_tok': ['8126', '의', '접시', '로', '해상도', '이름'],
  'sql': {'sel': 3, 'conds': [[0, 0, '8126']], 'agg': 0},
  'query': {'sel': 3, 'conds': [[0, 0, '8126']], 'agg': 0},
  'wvi_corenlp': [[0, 0]]},
 {'table_id': '2-1226461-1',
  'phase': 2,
  'question': 'brm p180을 섀시로 한 marlboro brm 팀의 최고 점수는?',
  'question_tok': ['brm',
   'p',
   '180',
   '을',
   '섀시',
   '로',
   '한',
   'marlboro',
   'brm',
   '팀',
   '의',
   '최고',
   '점수',
   '는',
   '?'],
  'sql': 

In [12]:
wemb_n, wemb_h, l_n, l_hpu, l_hs, \
nlu_tt, t_to_tt_idx, tt_to_t_idx \
                    = get_wemb_bert(bert_config, 
                                    model_bert, 
                                    tokenizer, 
                                    nlu_t, 
                                    hds, 
                                    max_seq_length,
                                    num_out_layers_n=num_target_layers, 
                                    num_out_layers_h=num_target_layers, 
                                    device=device)

NameError: name 'nlu_t' is not defined